In [1]:
from gurobipy import *
from prettytable import PrettyTable
import numpy as np

In [2]:
EC = Model('Dynamic Economic Dispatch')

In [3]:
#Sets
G = 4
Gen = range(G)
T =  3# Demand per Period
Period = range(T)

GenAt = {Gen[0]:{'a':0.12 ,'b':14.80 ,'c':89, 'd':1.2 ,'e':-5 ,'f':3 , 'Pmin':28 ,'Pmax':200,'RU0':40,'RD0':40 },
      Gen[1]:{'a':0.17 ,'b':16.57 ,'c':83 , 'd':2.3 ,'e':-4.24 ,'f':6.09 , 'Pmin':20 ,'Pmax':290,'RU0':30,'RD0':30  },
      Gen[2]:{'a':0.15 ,'b':15.55 ,'c':100 , 'd':1.1 ,'e':-2.15 ,'f':5.69 ,'Pmin':30 ,'Pmax':190,'RU0':30,'RD0':30  },
      Gen[3]:{'a':0.19 ,'b':16.21 ,'c':70 , 'd':1.1 ,'e':-3.99 ,'f':6.2 ,'Pmin':20 ,'Pmax':260, 'RU0':50,'RD0':50 }  }

In [4]:
Demand = [500, 850, 600]

In [5]:
Pmin = [GenAt[i]['Pmin'] for i in Gen]
Pmax = [GenAt[i]['Pmax'] for i in Gen]
l = [(a,b) for a in Gen for b in Period]
GenMax = sum(GenAt[i]['Pmax'] for i in Gen)
RMargin =100 #max(GenAt[i]['Pmax'] for i in Gen )
GenMax

940

In [6]:
P = EC.addVars(l, lb = Pmin*T, ub = Pmax*T)
RU = EC.addConstrs((P[(i, j)] - P[(i, j-1)]  <= GenAt[i]['RU0']) for i in Gen for j in range(1,T))
RD = EC.addConstrs(P[(i, j-1)] - P[(i, j)]  <= GenAt[i]['RD0'] for i in Gen for j in range(1,T))
Reserve = EC.addConstrs( GenMax-sum(P[i,j] for i in Gen) >= RMargin for j in Period)
Load = EC.addConstrs((P.sum('*',j) >= Demand[j] for j in Period), name = 'DemandConst')
GenCost =quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i,j]+GenAt[i]['a']*P[i,j]*P[i,j] for i in Gen for j in Period) 
EC.setObjective(GenCost, GRB.MINIMIZE)

In [7]:
EC.setObjective(GenCost, GRB.MINIMIZE)
EC.Params.Method=-1
EC.setParam( 'OutputFlag', True )
EC.optimize()



status = EC.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    EC.computeIIS()
    EC.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 22 rows, 12 columns and 56 nonzeros
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  QObjective range [2e-01, 4e-01]
  Bounds range     [2e+01, 3e+02]
  RHS range        [3e+01, 9e+02]
Presolve time: 0.02s

Barrier solved model in 0 iterations and 0.02 seconds
Model is infeasible
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0260000e+03   2.070000e+02   0.000000e+00      0s

IIS computed: 2 constraints and 0 bounds
IIS runtime: 0.01 seconds


In [8]:
PVar = np.asarray([[P[i,j].x for i in Gen] for j in Period])
PVar

AttributeError: b"Unable to retrieve attribute 'x'"

In [ ]:
Heading1 = ['Period']
Heading2 = ['Gen' + str(i) for i in Gen]
Headings = Heading1 + Heading2
Results = PrettyTable(Headings)
for j in Period:
    lis1 = ['Period ' + str(j)]
    lis2 = list(PVar[j,:])
    lis = lis1 +lis2
    Results.add_row(lis)              
print(Results)

In [ ]:
EC.printStats()

In [ ]:
import matplotlib.pyplot as plt
D =  np.asarray(Demand)
D2 = D[np.newaxis]
Data = np.concatenate((PVar,D2.T), axis =1)
plt.plot(Data)
plt.axis([0,T, 0, 1000])
plt.show()

In [ ]:
sum(GenAt[i]['Pmax']for i in Gen)

In [ ]:
max([GenAt[i]['Pmax'] for i in Gen])